In [4]:
import sys
import numpy as np
from numpy import percentile as per
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Save Model Using Pickle
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import pickle
import s3fs

In [11]:


airline_df = pd.read_csv('s3://finalprojectbucketairlinedata/combined_csv.csv')

C:\Users\supra\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
# check DF 
airline_df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,ORIGIN,...,CANCELLATION_CODE,DIVERTED,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 32
0,2019,2,4,15,1,2019-04-15,AA,19805,AA,DFW,...,NaN,0.0,140.0,1192.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,2,4,16,2,2019-04-16,AA,19805,AA,DFW,...,NaN,0.0,141.0,1192.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,2,4,17,3,2019-04-17,AA,19805,AA,DFW,...,NaN,0.0,132.0,1192.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,2,4,18,4,2019-04-18,AA,19805,AA,DFW,...,NaN,0.0,147.0,1192.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,2,4,19,5,2019-04-19,AA,19805,AA,DFW,...,NaN,0.0,143.0,1192.0,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# creating groups for carrier delay average
airline_df_group =airline_df.groupby(['MONTH','DAY_OF_MONTH','OP_UNIQUE_CARRIER'], as_index=False)['CARRIER_DELAY'].mean()
airline_df_group

,MONTH,DAY_OF_MONTH,OP_UNIQUE_CARRIER,CARRIER_DELAY
0,1,1,9E,18.246667
1,1,1,AA,25.590410
2,1,1,AS,17.211039
3,1,1,B6,30.907609
4,1,1,DL,30.531313
...,...,...,...,...
6368,12,31,OO,48.871940
6369,12,31,UA,22.153543
6370,12,31,WN,19.370173
6371,12,31,YV,39.967391


In [22]:
# ONE hot reformat
# Step 1: Label-encode data set
op_reformat = airline_df_group.OP_UNIQUE_CARRIER
label_encoder = LabelEncoder()
label_encoder.fit(op_reformat)
encoded_y = label_encoder.transform(op_reformat)
encoded_y

array([ 0,  1,  2, ..., 15, 16, 17])

In [23]:
# step 2: check for values and DF 
op_reformat.unique()

array(['9E', 'AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'G4', 'HA', 'MQ', 'NK',
       'OH', 'OO', 'QX', 'UA', 'WN', 'YV', 'YX'], dtype=object)

In [25]:
hacha_df = pd.DataFrame(encoded_y,op_reformat)
hacha_df.head(20)

,0
OP_UNIQUE_CARRIER,
9E,0
AA,1
AS,2
B6,3
DL,4
EV,5
F9,6
G4,7
HA,8


In [27]:
# step3 : need to create new column for labels to use on machine learning

conditions = [(airline_df_group['OP_UNIQUE_CARRIER']== '9E'),#1
            (airline_df_group['OP_UNIQUE_CARRIER']== 'AA'),#2
             (airline_df_group['OP_UNIQUE_CARRIER']== 'AS'),#3
             (airline_df_group['OP_UNIQUE_CARRIER']== 'B6'),#4
             (airline_df_group['OP_UNIQUE_CARRIER']== 'DL'),#5
            (airline_df_group['OP_UNIQUE_CARRIER']== 'EV'),#6
             (airline_df_group['OP_UNIQUE_CARRIER']== 'F9'),#7
             (airline_df_group['OP_UNIQUE_CARRIER']== 'G4'),#8
             (airline_df_group['OP_UNIQUE_CARRIER']== 'HA'),#9
             (airline_df_group['OP_UNIQUE_CARRIER']== 'MQ'),#10
             (airline_df_group['OP_UNIQUE_CARRIER']== 'NK'),#11
             (airline_df_group['OP_UNIQUE_CARRIER']== 'OH'),#12
             (airline_df_group['OP_UNIQUE_CARRIER']== 'OO'),#13
             (airline_df_group['OP_UNIQUE_CARRIER']== 'QX'),#14
             (airline_df_group['OP_UNIQUE_CARRIER']== 'UA'),#15
             (airline_df_group['OP_UNIQUE_CARRIER']== 'WN'),#16
             (airline_df_group['OP_UNIQUE_CARRIER']== 'YV'),#17
             (airline_df_group['OP_UNIQUE_CARRIER']== 'YX')]#18


values = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']

airline_df_group['labenc_OP_UNIQUE_CARRIER'] = np.select(conditions,values)
airline_df_group

,MONTH,DAY_OF_MONTH,OP_UNIQUE_CARRIER,CARRIER_DELAY,labenc_OP_UNIQUE_CARRIER
0,1,1,9E,18.246667,0
1,1,1,AA,25.590410,1
2,1,1,AS,17.211039,2
3,1,1,B6,30.907609,3
4,1,1,DL,30.531313,4
...,...,...,...,...,...
6368,12,31,OO,48.871940,12
6369,12,31,UA,22.153543,14
6370,12,31,WN,19.370173,15
6371,12,31,YV,39.967391,16


In [35]:
# assign x and y to values
X = airline_df_group[["MONTH","DAY_OF_MONTH","labenc_OP_UNIQUE_CARRIER"]]
y = airline_df_group["CARRIER_DELAY"]

In [36]:
# training module

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 47, test_size = 0.20)

In [37]:
# assign model to lineregression()
model = LinearRegression()

In [38]:
# model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [39]:
# Use our model to make predictions
predicted = model.predict(X_test)

# Score the predictions with mse and r2
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

Mean Squared Error (MSE): 94.07760784775273
R-squared (R2 ): 0.01199932297100259


In [41]:
pickle.dump(model,open('model.pkl','wb'))

In [42]:
model = pickle.load(open('model.pkl','rb'))
print(model.predict([[4, 3, 5]]))

[23.77684339]
